# What is a Plugin?

A Plugin allows you to interact with your Nanome application via a set of hooks and callbacks.

Existing documentation:
https://docs.nanome.ai/plugins/overview.html#nanome-stacks-plugins-and-how-to-use-them

There's two main classes we need to be concerned with right now.
<ul>
    <li><code>Plugin</code>: Handles Connections to NTS/ Low level packet stuff</li>
    <li><code>PluginInstance</code>: Collections of hooks and actions to interact with your Nanome Session</li>
</ul>





In [ ]:
# Get a simple plugin running 

from nanome.api import Plugin, PluginInstance
from nanome.util import Logs

class HelloNanomePlugin(PluginInstance):
    
    def on_run(self):
        Logs.message("Hello Nanome!")

# Information describing the plugin
name = 'Hello Nanome'
description = "Plugin that currently doesn't do much"
category = 'Demo'
has_advanced = False

# Create Plugin, and attach specific PluginInstance to it.
plugin = Plugin(name, description, category, has_advanced)
plugin.set_plugin_class(HelloNanomePlugin)

# Uncomment this to run
# plugin.run()

# Available Plugin Actions

Neat! We have a basic plugin, but what can we make it do?

Up to date details can be found here.
https://nanome.readthedocs.io/en/latest/nanome.api.plugin_instance.html

## Summary:
### Event Handlers
- <code>start</code>: Called when user “Activates” the plugin
- <code>update</code>: Called when when instance updates (multiple times per second)
- <code>on_run</code>: Called when user presses "Run"
- <code>on_stop</code>: Called when user disconnects or plugin crashes
- <code>on_advanced_settings</code>: Called when user presses "Advanced Settings"
- <code>on_complex_added</code>: Called whenever a complex is added to the workspace.
- <code>on_complex_removed</code>: Called whenever a complex is removed from the workspace.
- <code>on_presenter_changed</code>: Called when room's presenter changes.       
    
### Spatial Actions
- <code>zoom_on_structures</code>: Repositions and resizes the workspace such that the provided structure(s) will be in the center of the users view.</li>
- <code>center_on_structures</code>: Repositions the workspace such that the provided structure(s) will be in the center of the world.</li>
- <code>request_presenter_info</code>: Requests presenter account info (unique ID, name, email)</li>
- <code>request_controller_transforms</code>: Requests presenter controller info (head position, head rotation, left controller position, left controller rotation, right controller position, right controller rotation)</li>

 
### IO/Streaming

- <code>save_files</code>: Save files on the machine running Nanome, and returns result
- <code>create_writing_stream</code>: Create a stream allowing to continuously update properties of many objects
- <code>create_reading_stream</code>: Create a stream allowing to continuously receive properties of many objects
- <code>open_url</code>: Opens a URL in Nanome's computer browser
- <code>send_files_to_load</code>: Send file(s) to Nanome to load directly using Nanome's importers.
- <code>request_export</code>: Request a file export using Nanome exporters
- <code>set_plugin_list_button</code>: Set text and/or usable state of the buttons on the plugin connection menu in Nanome


### Workspace/Molecule Actions
- <code>request_workspace</code>: Request the entire workspace, in deep mode
- <code>add_to_workspace</code>: Add a list of complexes to the current workspace
- <code>request_complex_list</code>: Request the list of all complexes in the workspace, in shallow mode
- <code>request_complexes</code>: Requests a list of complexes by their indices
- <code>update_workspace</code>:  Replace the current workspace in the scene by the workspace in parameter
- <code>send_notification</code>: Send a notification to the user
- <code>update_structures_deep</code>: Update the specific molecular structures in the scene to match the structures in parameter.
- <code>update_structures_shallow</code>: Update the specific molecular structures in the scene to match the structures in parameter
- <code>apply_color_scheme</code>: Apply a color scheme to selected atoms.
 
### Menus/Stacks
- <code>update_menu</code>:  Update the menu in Nanome
- <code>update_content</code>: Update specific UI elements (button, slider, list...)
- <code>update_node</code>: Updates layout nodes and their children
- <code>set_menu_transform</code>: Update the position, scale, and rotation of the menu
- <code>request_menu_transform</code>: Requests spacial information of the plugin menu (position, rotation, scale)


### Calculations
- <code>add_bonds</code>: Calculate bonds
- <code>add_dssp</code>:  Use DSSP to calculate secondary structures
- <code>add_volume</code>: ???



In [ ]:
# Let's build a plugin that can load a complex from an external file
import requests
import tempfile

from nanome.api import Plugin, PluginInstance
from nanome.api.structure import Complex
from nanome.util import Logs


class ComplexLoaderPlugin(PluginInstance):
    
    def on_run(self):
        # Load sample complex into memory
        complex = create_complex()

        # This sends a request to NTS to add complex to the workspace.
        # When it is done processing, the callback function is executed
        self.add_to_workspace([complex], self.callback_fn)
        
    def callback_fn(self):
        Logs.message("Complex has been added to the Workspace!")
        
    def create_complex(self):
        """Download PDB file, and return as Complex object."""
        pdb_string = requests.get('https://files.rcsb.org/view/6Z5Q.pdb').content.decode()
        new_complex = Complex().io.from_pdb(string=pdb_string)
        return new_complex
    
# Information describing the plugin
name = 'ComplexLoader'
description = "Loads a Complex from an external PDB file into your Nanome workspace."
category = 'Demo'
has_advanced = False
plugin = Plugin(name, description, category, has_advanced)
plugin.set_plugin_class(ComplexLoaderPlugin)


# Uncomment this to run
# plugin.run()

# Adding menus to your Plugin

Oh yeah, that's looking tasty already! But we can really spice this up with some menus for the user to interact with in VR.

Menus are spatial objects that can be used to provide inputs for an operation. Your standard buttons, sliders, text input, etc.<br>
For details, view our [ReadThedocs](https://nanome.readthedocs.io/en/latest/ui.html)

Menu layouts are ideally designed inside of [Stack Studio](https://docs.nanome.ai/plugins/stackstudio.html). This will export a JSON representation of your menu, which can then be loaded through the Plugin.

<h1>Menus</h1>
Let's create a simple menu with two buttons, each of which will load a different pdb file.
This is what the menu should look like.

<div>
<img src="complex_loader_menu.png" width="250" align="left"/>
</div>





In [ ]:
import nanome


MENU_PATH = 'complex_loader_menu.json'

class ComplexLoaderMenu:
    """Wrapper class for interacting with menu."""
    def __init__(self, plugin):
        self.plugin = plugin
        self.create_menu()

    def create_menu(self):
        self.menu = nanome.ui.Menu.io.from_json(MENU_PATH)
        root = self.menu.root    
        # Buttons named in JSON file, we need to register callbacks
        self.btn_6z43 = root.find_node("btn_6Z43", True).get_content()
        self.btn_6z5q = root.find_node("btn_6Z5Q", True).get_content()
        self.btn_6z43.register_pressed_callback(self.load_complex_6z43)
        self.btn_6z5q.register_pressed_callback(self.load_complex_6z5q)
    
    def load_complex_6z43(self):
        rcsb_code = '6Z43'
        complex = self.load_complex(rcsb_code)
        self.plugin.add_to_workspace([complex])
    
    def load_complex_6z5q(self):
        rcsb_code = '6Z5Q'
        complex = self.load_complex(rcsb_code)
        self.plugin.add_to_workspace([complex])

    def load_complex(self, rcsb_code):
        url = f'https://files.rcsb.org/view/{rcsb_code}.pdb'
        pdb_string = requests.get(url).content.decode()
        new_complex = Complex().io.from_pdb(string=pdb_string)
        return new_complex

    def show_menu(self):
        self.menu.enabled = True
        self.plugin.update_menu(self.menu)

class ComplexLoaderPlugin(nanome.PluginInstance):
    def start(self):
        m = ComplexLoaderMenu()
        m.menu.index = i # Setting an index allows us to find the menu later.
        self.update_menu(m)
        m.show_menu()

plugin = nanome.Plugin("ComplexLoader", "Contents may be explosive", "LOL", False)
plugin.set_plugin_class(Plugin)
# plugin.run()

